In [31]:
import pandas as pd
import requests
import gzip
import json
import io
import numpy as np
import re

Read in Vermont reviews data and convert to dataframe.

In [32]:
url = 'https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/review-Vermont_10.json.gz'

response = requests.get(url, stream = True)
response.raise_for_status() 

with gzip.GzipFile(fileobj = io.BytesIO(response.content), mode = 'rb') as gz_file:
    data_list = [json.loads(line) for line in gz_file]

vt = pd.DataFrame(data_list)

Reviews dataframe has 324725 observations and 8 columns.

In [33]:
print(vt.shape)
print(vt.columns)

(324725, 8)
Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id'], dtype='object')


Read in business metadata and convert to dataframe.

In [34]:
url_metadata = 'https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/meta-Vermont.json.gz'

response_metadata = requests.get(url_metadata, stream = True)
response_metadata.raise_for_status() 

with gzip.GzipFile(fileobj = io.BytesIO(response_metadata.content), mode = 'rb') as gz_file:
    data_list1 = [json.loads(line) for line in gz_file]

vt_metadata = pd.DataFrame(data_list1)

Reviews dataframe has 11291 observations and 15 columns/

In [35]:
print(vt_metadata.shape)
print(vt_metadata.columns)

(11291, 15)
Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
       'state', 'relative_results', 'url'],
      dtype='object')


Data Cleaning

In reviews dataframe:
- Drop irrelavant 'name' column 
- Clean text and drop duplicate reviews


In [36]:
vt = vt.drop('name', axis = 1)
vt.columns = vt.columns.str.lower() 
vt['text'] = vt['text'].str.replace(r'\s+', ' ', regex = True)
vt = vt.drop_duplicates(subset = ['user_id', 'text', 'gmap_id', 'time'])

- Collapse 'pics' column into single array format and rename as 'pics_collapsed', drop original 'pics' column

In [37]:
def collapse_pics(pic_list):
    if not pic_list:
        return []  
    urls = []
    for pic_dict in pic_list:
        urls.extend(pic_dict.get('url', []))
    return urls

vt['pics_collapsed'] = vt['pics'].apply(collapse_pics)
vt = vt.drop('pics', axis = 1)

- Collapse 'resp' column into single array format and rename as 'resp_collapsed', drop original 'resp' column

In [38]:
def extract_texts(resp_entry):
    if isinstance(resp_entry, dict):
        # single response dict
        return resp_entry.get("text", "")
    elif isinstance(resp_entry, list):
        # list of response dicts
        return " ".join([d.get("text", "") for d in resp_entry if isinstance(d, dict)])
    elif isinstance(resp_entry, str):
        # fallback: extract with regex if it's a string
        texts = re.findall(r'"text":\s*"([^"]*)"', resp_entry)
        return " ".join(texts)
    else:
        return ""
    
vt["resp_collapsed"] = vt["resp"].apply(extract_texts)
vt['resp_collapsed'] = vt['resp_collapsed'].str.replace(r'\s+', ' ', regex = True)
vt = vt.drop("resp", axis = 1)

In [39]:
print(vt.head(3))

                 user_id           time  rating  \
0  118026874392842649478  1620085852324       5   
1  101532740754036204131  1580309946474       5   
2  115404122636203550540  1605195974445       5   

                                                text  \
0      Always done right from wood stove to screens!   
1  A great company to work with. Their sales and ...   
2  Great place to do business with staff was grea...   

                                 gmap_id pics_collapsed  \
0  0x89e02445cb9db457:0x37f42bff4edf7a43             []   
1  0x89e02445cb9db457:0x37f42bff4edf7a43             []   
2  0x89e02445cb9db457:0x37f42bff4edf7a43             []   

                                      resp_collapsed  
0  Good Evening, Rebecca! Thanks SO much for the ...  
1  Good Afternoon, Peter - Really appreciate the ...  
2  Hi Chad! Thank you so much for the 5-Star revi...  


In metadata dataframe:
- Drop irrelavant columns
- Clean text and drop duplicate observations

In [40]:
vt_metadata = vt_metadata.drop(['address', 'latitude', 'longitude', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC', 'state', 'relative_results'], axis = 1)
vt_metadata.columns = vt_metadata.columns.str.lower() 
vt_metadata['description'] = vt_metadata['description'].str.replace(r'\s+', ' ', regex = True)
vt_metadata['category'] = vt_metadata['category'].str.replace(r'\s+', ' ', regex = True)
vt_metadata = vt_metadata.drop_duplicates(subset = ['name','gmap_id', 'gmap_id'])

In [41]:
print(vt_metadata.head(3))

                       name                                gmap_id  \
0               Royal Group  0x89e02445cb9db457:0x37f42bff4edf7a43   
1  Foxglove Farm and Forest  0x4cb549e8877cf0d7:0xe8f003e6d73392ae   
2              Carr's Gifts  0x4cb54a301f3518f7:0x39af4eda1efb9117   

  description  category                                                url  
0        None       NaN  https://www.google.com/maps/place//data=!4m2!3...  
1        None       NaN  https://www.google.com/maps/place//data=!4m2!3...  
2        None       NaN  https://www.google.com/maps/place//data=!4m2!3...  


Save cleaned metadata dataframe as csv file.

In [42]:
vt_metadata.to_csv('vt_metadata.csv')

Scraping Process ...
- scrape our own data to fill in blanks for description
- scrape our own data to fill in blanks for category

Merge the reviews and metadata dataframes by common key 'gmap_id', save as vt_merged and drop 'gmap_id' column. Insert new column called 'review_id' such that each review has a unique id.

In [43]:
vt_merged = pd.merge(vt, vt_metadata, on = 'gmap_id', how = 'inner')
vt_merged = vt_merged.drop('gmap_id', axis = 1)
vt_merged['review_id'] = range(len(vt_merged))
column = vt_merged.pop('review_id')
vt_merged.insert(0, 'review_id', column)

Add new columns for category classification and final relevancy score.

In [47]:
vt_merged['is_image_ad'] = None
vt_merged['is_image_ad'] = vt_merged['is_image_ad'].astype(bool)

vt_merged['is_image_irrelevant'] = None
vt_merged['is_image_irrelevant'] = vt_merged['is_image_irrelevant'].astype(bool)

vt_merged['is_text_ad'] = None
vt_merged['is_text_ad'] = vt_merged['is_text_ad'].astype(bool)

vt_merged['is_text_irrelevant'] = None
vt_merged['is_text_irrelevant'] = vt_merged['is_text_irrelevant'].astype(bool)

vt_merged['is_text_rant'] = None
vt_merged['is_text_rant'] = vt_merged['is_text_rant'].astype(bool)

vt_merged['final_score'] = ""

print(vt_merged.columns)

Index(['review_id', 'user_id', 'time', 'rating', 'text', 'pics_collapsed',
       'resp_collapsed', 'name', 'description', 'category', 'url',
       'is_image_ad', 'is_image_irrelevant', 'is_text_ad',
       'is_text_irrelevant', 'is_text_rant', 'final_score'],
      dtype='object')


In [48]:
vt_merged.to_csv('vt_merged.csv')

In [46]:
sample_input = vt_merged.head(5)
sample_input.to_csv('sample_input.csv')